# Aplicación GraphQL con FastAPI y Strawberry usando Poetry y Docker

Este tutorial te guiará a través de la creación de una aplicación GraphQL en Python utilizando FastAPI y Strawberry, gestionando las dependencias con Poetry y, **novedad**, ejecutándola dentro de un contenedor Docker. Esto asegura un entorno consistente y reproducible para tu API de Machine Learning Ops.

### Objetivo

Aprenderás a:
* Configurar un proyecto Python con **Poetry**.
* Implementar una API GraphQL con **FastAPI** y **Strawberry**.
* **Contenerizar tu aplicación con Docker** para un despliegue y desarrollo consistentes.
* Interactuar con tu API GraphQL.

## 1. Configuración del Proyecto con Poetry

**Poetry** es una herramienta moderna para la gestión de dependencias y entornos Python. Nos ayudará a preparar nuestro proyecto de manera robusta para Docker.

### Instala Poetry (si no lo tienes):
Si aún no tienes Poetry instalado, la forma recomendada es a través de su instalador oficial:

* **En macOS / Linux / WSL (Windows Subsystem for Linux):**
  Abre tu terminal y ejecuta:

In [ ]:
# !curl -sSL https://install.python-poetry.org | python3 -

Retrieving Poetry metadata

The latest version (2.1.4) is already installed.


* **En Windows (PowerShell):**
  Abre PowerShell y ejecuta:

In [ ]:
# (Invoke-WebRequest -Uri https://install.python-poetry.org -UseBasicParsing).Content | python -

Verifica la instalación:

In [1]:
!poetry --version

zsh:1: command not found: poetry


### Crea un nuevo proyecto con Poetry:
Abre tu terminal y ejecuta:

In [2]:
# Crea el directorio del proyecto y navega hacia él
!mkdir python-graphql-docker-tutorial
%cd python-graphql-docker-tutorial

# Inicializa el proyecto Poetry (crea pyproject.toml)
!/Users/alejandrolloveras/.local/bin/poetry init --no-interaction # --no-interaction para no pedir confirmaciones en notebook

mkdir: python-graphql-docker-tutorial: File exists
/Users/alejandrolloveras/Documents/ESTUDIO/UBA/Materias/MLOps2/Repo/MLOPs2_2025_UBA/clase3/Practica/python-graphql-docker-tutorial
A pyproject.toml file with a project and/or a poetry section already exists.


### Añade las dependencias:
Añade las librerías necesarias. **Importante:** Asegúrate de que el rango de Python en tu `pyproject.toml` sea `python = ">=3.11,<4.0"` para evitar conflictos de resolución con `strawberry-graphql`.

In [3]:
# Primero, ajusta tu pyproject.toml manualmente si es necesario:
# Abre el archivo pyproject.toml en tu editor y asegúrate de que tenga:
# [tool.poetry.dependencies]
# python = ">=3.11,<4.0"

!/Users/alejandrolloveras/.local/bin/poetry add fastapi uvicorn strawberry-graphql requests

Using version ^0.116.1 for fastapi
Using version ^0.35.0 for uvicorn
Using version ^0.282.0 for strawberry-graphql
Using version ^2.32.5 for requests

Updating dependencies
Resolving dependencies... (2.5s)Resolving dependencies... (1.7s)Resolving dependencies... (2.3s)

Package operations: 23 installs, 0 updates, 0 removals

  - Installing idna (3.10): Pending...
  - Installing sniffio (1.3.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing sniffio (1.3.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing idna (3.10): Downloading... 0%
  - Installing sniffio (1.3.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing sniffio (1.3.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing idna (3.10): Downloading... 100%
  - Installing sniffio (1.3.1): Pending...
  - Installing typing-extensions (4.15.0): Pending...
  - Installing sniffio (1.3.1): Pending...
  - Installi

### Crea el archivo `app.py`:
Este será el código de nuestra API GraphQL. Crea el archivo `app.py` en la raíz de tu proyecto.

In [4]:
%%writefile app.py
import strawberry
from fastapi import FastAPI
from typing import List, Optional

# --- Datos simulados en memoria ---
class BookData:
    def __init__(self, id: str, title: str, author: str, year: Optional[int] = None):
        self.id = id
        self.title = title
        self.author = author
        self.year = year

mock_books = [
    BookData("b1", "El principito", "Antoine de Saint-Exupéry", 1943),
    BookData("b2", "Cien años de soledad", "Gabriel García Márquez", 1967),
    BookData("b3", "1984", "George Orwell", 1949),
]
next_book_id = len(mock_books) + 1

# --- Definición del Esquema GraphQL ---
@strawberry.type
class Book:
    id: str
    title: str
    author: str
    year: Optional[int] = None

@strawberry.type
class Query:
    @strawberry.field
    def hello(self) -> str:
        return "¡Hola desde tu API GraphQL en Docker!"

    @strawberry.field
    def books(self) -> List[Book]:
        return mock_books

    @strawberry.field
    def book(self, id: str) -> Optional[Book]:
        for book in mock_books:
            if book.id == id:
                return book
        return None

@strawberry.input
class CreateBookInput:
    title: str
    author: str
    year: Optional[int] = None

@strawberry.type
class Mutation:
    @strawberry.mutation
    def create_book(self, data: CreateBookInput) -> Book:
        global next_book_id
        new_id = f"b{next_book_id}"
        next_book_id += 1
        new_book = BookData(new_id, data.title, data.author, data.year)
        mock_books.append(new_book)
        return new_book

    @strawberry.mutation
    def delete_book(self, id: str) -> Optional[Book]:
        global mock_books
        original_len = len(mock_books)
        mock_books = [book for book in mock_books if book.id != id]
        if len(mock_books) < original_len:
            return BookData(id, "Eliminado", "N/A") 
        return None

schema = strawberry.Schema(query=Query, mutation=Mutation)

# 6. Integrar GraphQL con FastAPI
app = FastAPI(
    title="Mi API GraphQL de Libros",
    description="Una API GraphQL de ejemplo en Docker.",
    version="1.0.0",
)

app.add_route("/graphql", strawberry.asgi.GraphQL(schema), name="graphql")

@app.get("/status")
async def get_status() -> dict:
    return {"status": "running", "api_type": "GraphQL & REST (Hybrid)"}


Writing app.py


### Crea el archivo `Dockerfile`:
Este archivo contiene las instrucciones para construir tu imagen Docker. Crea el archivo `Dockerfile` en la raíz de tu proyecto (junto a `app.py` y `pyproject.toml`).

In [19]:
%%writefile Dockerfile
# Usa una imagen base de Python oficial
FROM python:3.11-slim-bookworm

# Establece el directorio de trabajo dentro del contenedor
WORKDIR /app

# Instala Poetry
RUN pip install poetry

# Copia los archivos de configuración de Poetry
COPY pyproject.toml poetry.lock* ./

# Instala las dependencias de producción usando Poetry
# RUN poetry install --no-root --no-dev --no-interaction
RUN poetry install --no-root --only main --no-interaction

# Copia el código de tu aplicación
COPY app.py ./

# Expone el puerto que usará Uvicorn
EXPOSE 8000

# Comando para ejecutar la aplicación con Uvicorn
# 'poetry run' asegura que uvicorn se ejecute dentro del entorno virtual gestionado por Poetry
CMD ["poetry", "run", "uvicorn", "app:app", "--host", "0.0.0.0", "--port", "8000"]

Overwriting Dockerfile


## 2. Construyendo y Ejecutando el Servidor Docker

Ahora construiremos la imagen Docker de tu aplicación y la ejecutaremos como un contenedor.

### Construye la imagen Docker:
Abre tu terminal en el directorio raíz de tu proyecto (`python-graphql-docker-tutorial`) y ejecuta:

In [21]:
!docker build -t graphql-app .

[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 816B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.11-slim-bookw  0.2s
[+] Building 0.3s (1/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 816B                                       0.0s
 => [internal] load metadata for docker.io/library/python:3.11-slim-bookw  0.3s
[+] Building 0.5s (1/2)                                    docker:desktop-linux
 => [internal] l

### Ejecuta el contenedor Docker:
Ejecuta el siguiente comando para iniciar tu aplicación GraphQL dentro de un contenedor. La opción `-d` lo ejecuta en segundo plano, y `-p 8000:8000` mapea el puerto 8000 del contenedor al puerto 8000 de tu máquina local.

In [22]:
!docker run -d --name graphql-server -p 8000:8000 graphql-app
# Para detener el servidor Docker más tarde:
# !docker stop graphql-server
# !docker rm graphql-server

969b22f3475a190dbfe8b6b9b009271f4ccbfebd4ff7af5a654e4f637b79c84c


## 3. Interactuando con tu API GraphQL

Ahora que tu servidor GraphQL está corriendo en Docker, puedes interactuar con él.

### A. Usando GraphiQL (Interfaz interactiva en el navegador)

1.  Abre tu navegador web y ve a `http://localhost:8000/graphql`.


### B. Usando un Cliente Python (`requests`)

Puedes usar el mismo script `client.py` que creamos antes, ya que el endpoint (URL) sigue siendo el mismo (`http://localhost:8000/graphql`).

In [ ]:
%%writefile client.py
import requests
import json
from typing import Optional

graphql_endpoint = "http://localhost:8000/graphql"

def execute_graphql_query(query: str, variables: Optional[dict] = None) -> dict:
    """Función para ejecutar consultas/mutaciones GraphQL."""
    payload = {"query": query}
    if variables:
        payload["variables"] = variables

    try:
        response = requests.post(graphql_endpoint, json=payload)
        response.raise_for_status() # Lanza un error para códigos 4xx/5xx
        return response.json()
    except requests.exceptions.ConnectionError:
        print("Error: No se pudo conectar con el servidor. Asegúrate de que el contenedor Docker esté corriendo.")
        return {"errors": [{"message": "Connection error"}]}
    except requests.exceptions.HTTPError as http_err:
        print(f"Error HTTP: {http_err} - {response.text}")
        return {"errors": [{"message": f"HTTP error: {http_err}"}]}
    except Exception as err:
        print(f"Ocurrió otro error: {err}")
        return {"errors": [{"message": f"An unexpected error occurred: {err}"}]}


print("--- Cliente Python para API GraphQL ---")

# --- Query de ejemplo: obtener todos los libros, solo título y autor ---
print("\n--- Consultando todos los libros (solo título y autor) ---")
all_books_query = """
query GetBooks {
  books {
    title
    author
  }
}
"""
result = execute_graphql_query(all_books_query)
if "data" in result:
    for book in result["data"]["books"]:
        print(f"- {book['title']} por {book['author']}")
elif "errors" in result:
    print(f"Errores: {result['errors']}")

# --- Mutation de ejemplo: crear un nuevo libro ---
print("\n--- Creando un nuevo libro ---")
create_book_mutation = """
mutation AddBook($title: String!, $author: String!, $year: Int) {
  createBook(data: {title: $title, author: $author, year: $year}) {
    id
    title
    author
    year
  }
}
"""
variables = {
    "title": "La sombra del viento",
    "author": "Carlos Ruiz Zafón",
    "year": 2001
}
result = execute_graphql_query(create_book_mutation, variables)
if "data" in result:
    new_book = result["data"]["createBook"]
    print(f"Libro creado: ID={new_book['id']}, Título='{new_book['title']}'")
elif "errors" in result:
    print(f"Errores: {result['errors']}")

# --- Query de verificación: obtener todos los libros de nuevo para ver el nuevo ---
print("\n--- Verificando todos los libros después de la creación ---")
result = execute_graphql_query(all_books_query)
if "data" in result:
    for book in result["data"]["books"]:
        print(f"- {book['title']} por {book['author']}")
elif "errors" in result:
    print(f"Errores: {result['errors']}")

# --- Mutation de ejemplo: eliminar un libro ---
print("\n--- Eliminando un libro (por ejemplo, el b1) ---")
delete_book_mutation = """
mutation RemoveBook($id: String!) {
  deleteBook(id: $id) {
    id
    title
  }
}
"""
variables = {"id": "b1"}
result = execute_graphql_query(delete_book_mutation, variables)
if "data" in result and result["data"]["deleteBook"]:
    deleted_book = result["data"]["deleteBook"]
    print(f"Libro eliminado: ID={deleted_book['id']}, Título='{deleted_book['title']}'")
elif "errors" in result:
    print(f"Errores: {result['errors']}")
else:
    print("No se encontró el libro para eliminar o ya fue eliminado.")


print("\n--- Fin del ejemplo de cliente GraphQL ---")

2.  **Ejecuta el script del cliente:**
    Asegúrate de que el contenedor Docker `graphql-server` esté ejecutándose en segundo plano (`!docker run -d ...`). Luego, en una terminal, navega a tu directorio del proyecto (`python-graphql-docker-tutorial`) y ejecuta:

In [ ]:
poetry run python client.py

¡Felicidades! Has configurado y ejecutado una aplicación GraphQL completa con FastAPI, Strawberry, Poetry y Docker. Este es un setup robusto para cualquier API de MLOps.